In [1]:
# Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint

In [12]:
# Define URLs to be scraped
url_base = "http://www.ufcstats.com/statistics/fighters?char="
url_page = "&page=all"
url = "http://www.ufcstats.com/statistics/fighters?char=a"
url_with_page_all = "http://www.ufcstats.com/statistics/fighters?char=a&page=all"

In [13]:
# Send get request to URL provided
page = requests.get(url_with_page_all)

# Return the html text of the page
soup = bs(page.text, 'lxml')

# Find the table of data on the page 
table_body = soup.find('table')

In [14]:
print(table_body)

<table class="b-statistics__table">
<thead class="b-statistics__table-caption">
<tr class="b-statistics__table-row">
<th class="b-statistics__table-col">
          First
        </th>
<th class="b-statistics__table-col">
          Last
        </th>
<th class="b-statistics__table-col">
          Nickname
        </th>
<th class="b-statistics__table-col">
          Ht.
        </th>
<th class="b-statistics__table-col">
          Wt.
        </th>
<th class="b-statistics__table-col">
          Reach
        </th>
<th class="b-statistics__table-col">
          Stance
        </th>
<th class="b-statistics__table-col b-statistics__table-col_type_small">
          W
        </th>
<th class="b-statistics__table-col b-statistics__table-col_type_small">
          L
        </th>
<th class="b-statistics__table-col b-statistics__table-col_type_small">
          D
        </th>
<th class="b-statistics__table-col">
          Belt
        </th>
</tr>
</thead>
<tbody>
<tr class="b-statistics__table-r

In [15]:
# Empty lists to store data
table_data = []
current_row = []

# Find HTML where table data is listed
for row in table_body.find_all('tr'):
    cols = row.find_all('td')
    # Enumerate through each row/column
    for i, ele in enumerate(cols, start=1):
        col = ele.text.strip()
        current_row.append(col)

        # When i reaches 10, add current row to row_data and reset the current_row list
        if i == 10:
            table_data.append(current_row)
            current_row = []

# Convert data to a DataFrame using row_data
page_df = pd.DataFrame(table_data).drop(0, axis=1)

In [16]:
print(page_df.shape)
page_df.head()

(202, 10)


,1,2,3,4,5,6,7,8,9,10
0,Tom,Aaron,,--,155 lbs.,--,,5,3,0
1,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4,6,0
2,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28,4,0
3,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10,15,0
4,Hamdy,Abdelwahab,The Hammer,"6' 2""",264 lbs.,"72.0""",Southpaw,5,0,0


In [17]:
def get_fighter_details(table_body: bs):
    """Get fighter details from a single web page."""
    
    # Empty lists to store data
    table_data = []
    current_row = []

    # Find HTML where table data is listed
    for row in table_body.find_all('tr'):
        cols = row.find_all('td')
        # Enumerate through each row/column
        for i, ele in enumerate(cols, start=1):
            col = ele.text.strip()
            current_row.append(col)

            # When i reaches 10, add current row to row_data and reset the current_row list
            if i == 10:
                table_data.append(current_row)
                current_row = []

    # Convert data to a DataFrame using row_data
    page_df = pd.DataFrame(table_data).drop(0, axis=1)
    
    return page_df

In [24]:
# List of letters to be used to sort web page by last name
letters = [chr(i) for i in range(ord('a'), ord('z') + 1)]

# Master DataFrame to store all pages fighter data
master_df = pd.DataFrame()

# Loop to go through all web pages based on the letter of the last name
for letter in letters:
    # Set url to be scraped
    page_url = url_base + letter + url_page
    print(page_url)
    
    # Send get request to URL provided
    page = requests.get(page_url)

    # Return the html text of the page
    soup = bs(page.text, 'lxml')

    # Find the table of data on the page 
    table_body = soup.find('table')
    print(table_body)
    
    # Get fighter details from a single web page
    page_df = get_fighter_details(table_body)
    
    # Add page_data to master_df
    master_df = pd.concat([master_df, page_df], ignore_index=True, axis=0)

http://www.ufcstats.com/statistics/fighters?char=a&page=all
<table class="b-statistics__table">
<thead class="b-statistics__table-caption">
<tr class="b-statistics__table-row">
<th class="b-statistics__table-col">
          First
        </th>
<th class="b-statistics__table-col">
          Last
        </th>
<th class="b-statistics__table-col">
          Nickname
        </th>
<th class="b-statistics__table-col">
          Ht.
        </th>
<th class="b-statistics__table-col">
          Wt.
        </th>
<th class="b-statistics__table-col">
          Reach
        </th>
<th class="b-statistics__table-col">
          Stance
        </th>
<th class="b-statistics__table-col b-statistics__table-col_type_small">
          W
        </th>
<th class="b-statistics__table-col b-statistics__table-col_type_small">
          L
        </th>
<th class="b-statistics__table-col b-statistics__table-col_type_small">
          D
        </th>
<th class="b-statistics__table-col">
          Belt
        <

AttributeError: 'NoneType' object has no attribute 'find_all'

In [9]:
print(master_df.shape)
master_df.tail()


(650, 10)


,1,2,3,4,5,6,7,8,9,10
645,Mariya,Agapova,Demonslayer,"5' 6""",125 lbs.,"68.0""",Southpaw,10,4,0
646,Marcelo,Aguiar,,"5' 10""",170 lbs.,--,Orthodox,2,3,1
647,Fabio,Aguiar,,"6' 0""",185 lbs.,--,,17,1,0
648,Edwin,Aguilar,Tigre,"5' 10""",185 lbs.,--,,26,19,0
649,Jessica,Aguilar,Jag,"5' 3""",115 lbs.,"63.0""",Orthodox,20,8,0
